# Cài đặt thuật toán K-means phân tán với PySpark

- [Spark Machine Learning Library](#mllib)
- [Thuật toán K-means với PySpark](#spark_kmeans)
- [So sánh kết quả với thư viện Scikit-learn](#comparison)

## Spark Machine Learning Library <a name="mllib"/>

Spark cung cấp [thư viện MLLib (Machine Learning Library)](https://spark.apache.org/docs/3.1.1/ml-guide.html) cho phép triển khai các dự án machine learning phân tán. Các tính năng của MLLib bao gồm:
- Một số thuật toán ML tiêu biểu: phân lớp (classification), hồi qui (regression), phân cụm (clustering), collaborative filtering.
- Xử lý đặc trưng (featurization): trích xuất (feature extraction), biến đối (transformation), giảm chiều (dimensionality reduction), chọn lọc (feature selection).

- Pipelines: các công cụ xây dựng, tinh chỉnh và đánh giá mô hình ML.
- Các tiện ích: xử lý dữ liệu, lưu/nạp mô hình, v.v.

Bên cạnh thư viện MLLib (RDD-based API), hiện nay Spark cung cấp thư viện ML mới (DataFrame-based API) với nhiều tính năng dễ sử dụng hơn MLLib. 

## Thuật toán K-means với PySpark <a name="spark_kmeans"/>
### Khai báo các thư viện 

In [1]:
import pandas as pd
import numpy as np
from pyspark.ml.clustering import KMeans
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler

### Khởi tạo Spark Session

Spark phiên bản 2.0 trở đi cung cấp lớp SparkSession để khởi tạo các chức năng của Spark. Sau khi tạo Spark Session, người dùng có thể lập trình với RDD, DataFrame và Dataset. 

In [2]:
# Create new Spark session
spark = SparkSession.builder.appName("Distributed KMeans Example").getOrCreate()

22/05/08 16:44:49 WARN Utils: Your hostname, DESKTOP-HHU5MDD resolves to a loopback address: 127.0.1.1; using 192.168.1.77 instead (on interface wifi0)
22/05/08 16:44:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/08 16:44:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Tạo dữ liệu dạng DataFrame

Với Spark Context, người dùng phải tạo mới RDD sau đó lập trình xử lý trên RDD. Spark Session cung cấp các API xử lý với các loại dữ liệu khác, gồm cả DataFrame.

Để nạp dữ liệu iris vào Spark cluster, có thể thực hiện như sau: trước hết đọc dữ liệu đưa vào DataFrame dùng thư viện pandas, sau sử dụng phương thức spark.createDataFrame() để nạp dữ liệu vào Spark DataFrame.

In [3]:
iris_data = spark.createDataFrame(pd.read_csv('/mnt/d/Code/Big-Data/Lab5_DistributedMachineLearning/data/iris.data', 
                                                header=None, names=['sepal-length',
                                                                    'sepal-width', 'petal-length', 
                                                                    'petal-width','label']))
print("First 10 rows:")
iris_data.show(10)

IsADirectoryError: [Errno 21] Is a directory: '/mnt/d/Code/Big-Data/Lab5_DistributedMachineLearning/data'

### Tạo vector đặc trưng từ DataFrame

Để tạo vector đặc trưng (feature vector) cho từng dòng dữ liệu, dùng lớp VectorAssembler để ghép giá trị ở các cột trong DataFrame thành một cột mới chứa vector đặc trưng:

In [4]:
assembler = VectorAssembler(inputCols = ["sepal-length", "sepal-width", "petal-length", "petal-width"], 
                            outputCol="features") 
irisFeatures = assembler.transform(iris_data) 
irisFeatures.show(5)

+------------+-----------+------------+-----------+-----------+-----------------+
|sepal-length|sepal-width|petal-length|petal-width|      label|         features|
+------------+-----------+------------+-----------+-----------+-----------------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
+------------+-----------+------------+-----------+-----------+-----------------+
only showing top 5 rows



Lúc này dữ liệu đã sẵn sàng để đưa vào huấn luyện mô hình ML (ở đây là K-means).
Gỉa sử chọn số cụm là 3, tiến hành huấn luyện mô hình K-means với PySpark bằng các lệnh sau:

### Huấn luyện mô hình

In [5]:
# Huấn luyện mô hình K-means với K=3 trên toàn bộ tập dữ liệu.
# Trong thực tế, để đánh giá một mô hình ML, tập dữ liệu thường được chia làm 3 phần:
# - train set dùng để huấn luyện,
# - validation set dùng để đánh giá, tinh chỉnh mô hình trong quá trình huấn luyện,
# - test set để kiểm tra hiệu năng của mô hình trên dữ liệu hoàn toàn mới.
kmeans = KMeans().setK(3).setSeed(0)
model = kmeans.fit(irisFeatures)

### In kết quả

In [6]:
# In ra tâm điểm của các cụm

centers = model.clusterCenters()
np.set_printoptions(precision=2)

print("Centers learned by Spark ML: ")
for center in centers:
    print(center)

spark.stop()

Centers learned by Spark ML: 
[5.9  2.75 4.39 1.43]
[5.01 3.42 1.46 0.24]
[6.85 3.07 5.74 2.07]


### So sánh với kết quả chạy K-means bằng thư viện Scikit-learn <a name="comparison"/>

In [7]:
from sklearn import datasets
from sklearn.cluster import KMeans

np.random.seed(0)

In [8]:
# Nạp dữ liệu

iris = datasets.load_iris()
X = iris.data

kmeans = KMeans(n_clusters=3, random_state=0).fit(X)

print('Centers learned by scikit-learn:')
print(kmeans.cluster_centers_)

Centers learned by scikit-learn:
[[5.9  2.75 4.39 1.43]
 [5.01 3.43 1.46 0.25]
 [6.85 3.07 5.74 2.07]]
